## LangChain Tools

- 각 검색 서비스들이 LangChain에 쉽게 통합될 수 있도록 래핑되어있음
- [Tools](https://python.langchain.com/docs/integrations/tools/)
    - DuckDuckgoSearch
    - Google Serper
    - Tavily Search

In [ ]:
%pip install -U langchain-community ddgs tavily-python

In [ ]:
# duckduckgo-search
%pip install -U duckduckgo-search

In [8]:
from dotenv import load_dotenv
# OPEN AI API 키 설정
load_dotenv()

True

### [DuckDuckGo](https://duckduckgo.com/)

- 회원가입이 필요없는 검색엔진
- 검색 결과를 문자열로 받아 context에 포함하여 활용

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [3]:
template = """You are an AI assistant. Answer the question.
If you don't know the answer, just say you don't know.

Question: {question}
Answer:"""

prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(model="gpt-4")
parser = StrOutputParser()

chain = prompt | model | parser

C:\Users\USER\AppData\Local\Temp\ipykernel_18728\597195285.py:8: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI(model="gpt-4")


In [4]:
result = chain.invoke({"question": "Which BTS member was the last to go to the military?"})
print(result)

As of my last update, none of the BTS members have enlisted in the military yet. However, the group's management, Big Hit Entertainment, has stated that the members will fulfill their required service. The timing for each member will depend on various factors, including their age and the military's schedule.


In [5]:
## Adding Web Search Tools
from langchain.tools import DuckDuckGoSearchRun
from langchain_core.runnables import RunnablePassthrough

search = DuckDuckGoSearchRun()

## Provide "the latest context information" from web search
template = """Answer the question based on context.

Question: {question}
Context: {context}
Answer:"""

prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(model="gpt-4")
parser = StrOutputParser()

chain = (
    {"question": RunnablePassthrough(), "context": RunnablePassthrough()}
    | prompt 
    | model 
    | parser
)

In [6]:
# 검색 결과 확인
search_result = search.run("Which BTS members are still serving in the military as of May 2025?")

# langchain.tools의 DuckDuckgoSearchRun class는 아직 duckduckgo-search 패키지를 사용하고 있어, ddgs로 수정한 뒤 사용
print(search_result)

Apr 14, 2025 · The first member to embark in military service was Jin, who enlisted on December 13, 2023. A few months later, J-Hope was the second … Jun 23, 2025 · All seven members of BTS have officially completed their military service. In just one week, RM, V, Jung Kook and Jimin completed their … Apr 28, 2025 · With all seven members set to finish their mandatory military service by June 2025, their reunion will be more than just a comeback. Since … Jun 10, 2025 · Though each member varies in age, Big Hit confirmed that they are all expected to reunite as a group in 2025. Here is a list of when each BTS … Jan 2, 2025 · All members serving in the military will be discharged from the military and stand in front of fans in full. All BTS are serving in the military …


c:\Users\USER\anaconda3\envs\llm\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


In [7]:
question = "Which BTS members are still serving in the military as of May 2025?"

chain.invoke({"question": question, "context": search.run(question)})

c:\Users\USER\anaconda3\envs\llm\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
c:\Users\USER\anaconda3\envs\llm\Lib\site-packages\langchain_community\chat_models\openai.py:495: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()


'All seven members of BTS have officially completed their military service by May 2025.'

### [Tavily](https://www.tavily.com/)

- 검색엔진으로 맞춤형 LLM, RAG를 위한 정확하고 실시간 검색 결과를 제공
- 한 달에 1000회 요청 무료

In [9]:
from dotenv import load_dotenv
# TAVILY_API 키 설정
load_dotenv()

True

In [10]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [11]:
from langchain_community.tools import TavilySearchResults

search = TavilySearchResults(max_results=2)

C:\Users\USER\AppData\Local\Temp\ipykernel_18728\1722044080.py:3: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  search = TavilySearchResults(max_results=2)


In [12]:
query = "2025년 애플의 주가 전망에 대해서 분석하세요."
search_results = search.invoke(query)

In [14]:
search_results

[{'title': '시총 3조달러 돌파한 애플, 2025년 4조달러도 가능 > 시애틀 뉴스/핫이슈',
  'url': 'https://www.seattlen.com/hot/29561',
  'content': '◇ 애플은 안전자산: 또 애플의 지난 분기 매출과 순익이 감소했지만 시장의 예상은 상회했다. 꾸준한 자사주 매입과 함께 이같은 실적은 글로벌 경제 불확실성의 시기에 투자자들 사이에서 애플이 안전한 투자처라는 명성을 재확인했다.\n\n이에 따라 마감가 기준으로도 시총 3조 달러를 돌파했다. 올 들어 애플은 45% 급등했다. 이는 나스닥의 상승률 32%보다 높은 것이다.\n\n◇2025년 4조 달러 돌파할 것: 애플이 이에 그치지 않고 시총 4조 달러를 돌파할 것이란 전망도 나왔다.\n\n미국 웨드부시 증권의 유명 애널리스트 대니얼 아이브스는 애플에 대한 투자 의견을 \'매수\'로, 목표주가를 220달러로 제시했다.\n\n아이브스는 "월가는 애플의 성장 잠재력을 과소평가했다"며 "회의론자들은 2023년 애플 주식에 대해 성장 스토리가 깨졌다고 말했다"고 주장했다. [...] 지난달 30일(현지시간) 뉴욕증시에서 애플은 전거래일보다 2.31% 급등한 193.97 달러를 기록했다. 이로써 시총이 3조510억 달러를 기록, 마침내 마감가 기준으로도 시총 3조 달러를 돌파했다.\n\nImage 8: 애플 일일 주가추이 - 야후 파이낸스 갈무리\n애플 일일 주가추이 - 야후 파이낸스 갈무리\n\n애플은 그동안 장중 시총 3조 달러를 돌파한 적은 있으나 마감가 기준으로 시총 3조 달러를 돌파한 것은 이번이 처음이다.\n\n시총 3조 달러를 돌파한 기업은 인류역사상 애플이 처음이다.\n\n◇시티 매수 추천, 추가로 30% 랠리한다: 이는 전일 월가의 주요 투자은행인 시티가 애플에 대해 매수 추천을 했기 때문으로 풀이된다. 시티는 애플이 추가로 30% 상승할 여력이 있다며 \'매수\'를 추천했다.\n\n시티의 애플 담당 분석가 아티프 말릭은 애플에 매수 등급을 부여하고

In [13]:
for result in search_results:
    print(result)
    print("-" * 100)

{'title': '시총 3조달러 돌파한 애플, 2025년 4조달러도 가능 > 시애틀 뉴스/핫이슈', 'url': 'https://www.seattlen.com/hot/29561', 'content': '◇ 애플은 안전자산: 또 애플의 지난 분기 매출과 순익이 감소했지만 시장의 예상은 상회했다. 꾸준한 자사주 매입과 함께 이같은 실적은 글로벌 경제 불확실성의 시기에 투자자들 사이에서 애플이 안전한 투자처라는 명성을 재확인했다.\n\n이에 따라 마감가 기준으로도 시총 3조 달러를 돌파했다. 올 들어 애플은 45% 급등했다. 이는 나스닥의 상승률 32%보다 높은 것이다.\n\n◇2025년 4조 달러 돌파할 것: 애플이 이에 그치지 않고 시총 4조 달러를 돌파할 것이란 전망도 나왔다.\n\n미국 웨드부시 증권의 유명 애널리스트 대니얼 아이브스는 애플에 대한 투자 의견을 \'매수\'로, 목표주가를 220달러로 제시했다.\n\n아이브스는 "월가는 애플의 성장 잠재력을 과소평가했다"며 "회의론자들은 2023년 애플 주식에 대해 성장 스토리가 깨졌다고 말했다"고 주장했다. [...] 지난달 30일(현지시간) 뉴욕증시에서 애플은 전거래일보다 2.31% 급등한 193.97 달러를 기록했다. 이로써 시총이 3조510억 달러를 기록, 마침내 마감가 기준으로도 시총 3조 달러를 돌파했다.\n\nImage 8: 애플 일일 주가추이 - 야후 파이낸스 갈무리\n애플 일일 주가추이 - 야후 파이낸스 갈무리\n\n애플은 그동안 장중 시총 3조 달러를 돌파한 적은 있으나 마감가 기준으로 시총 3조 달러를 돌파한 것은 이번이 처음이다.\n\n시총 3조 달러를 돌파한 기업은 인류역사상 애플이 처음이다.\n\n◇시티 매수 추천, 추가로 30% 랠리한다: 이는 전일 월가의 주요 투자은행인 시티가 애플에 대해 매수 추천을 했기 때문으로 풀이된다. 시티는 애플이 추가로 30% 상승할 여력이 있다며 \'매수\'를 추천했다.\n\n시티의 애플 담당 분석가 아티프 말릭은 애플에 매수 등급을 부여하고 목표가로

### Serper API (Google)

- [Serper](https://serper.dev/)에서 회원가입
     - 초기 2500 크레딧 무료


In [15]:
from dotenv import load_dotenv
# SERPER_API 키 설정
load_dotenv()

True

In [16]:
from langchain_community.utilities import GoogleSerperAPIWrapper

# type 파라미터를 통해 news, images, places를 검색가능
search = GoogleSerperAPIWrapper()

In [17]:
search.run("Obama's first name?")

'Barack Obama: 44th U.S. President. Barack Hussein Obama II is an American politician who was the 44th president of the United States from 2009 to 2017. A member of the Democratic Party, he was the first African American president. Barack Obama Born: August 4, 1961 (age 64 years), Kapiʻolani Medical Center for Women & Children, Honolulu, HI. Barack Obama Vice president: Joe Biden (2009–2017). Barack Obama Party: Democratic Party. Barack Obama Presidential term: January 20, 2009 – January 20, 2017. Barack Obama Children: Malia Ann Obama and Sasha Obama. Barack Obama Parents: Stanley Ann Dunham and Barack Obama Sr.. Full Name of Father. BARACK. 9. HUSSEIN. OBAMA. 10. Age of Father. 11. Birthplace (Island, State ... Barack Hussein Obama II was born August 4, 1961, in Honolulu, Hawaii, to parents Barack H. Obama, Sr., and Stanley Ann Dunham. it specifically asks for "Full former name(s). Obama put “None”, when in fact he went by the name Barry Soetoro, and Barry Obama. It is further believ

In [18]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda 

search = GoogleSerperAPIWrapper()

llm = ChatOpenAI(model="gpt-4.1-nano")

prompt = ChatPromptTemplate.from_template(
    "다음 검색 결과를 한글로 요약해줘:\n{search_results}"
)

chain = {'search_results': RunnableLambda(search.run)} | prompt | llm

result = chain.invoke("2025년 인공지능 트렌드")
print(result.content)

2025년에는 인공지능(AI)이 기술 분야에서 가장 중요한 핵심 기술로 꼽히며, 예측형 AI, 생성형 AI, 기계학습, 자연어처리 등이 주도할 것으로 예상됩니다. AI와 예측 분석 기술은 마케터의 비즈니스 성장을 재정의하며 고객 니즈 예측과 성과 측정을 가능하게 합니다. 2024년에는 AI가 파일럿 단계에서 생산 단계로 넘어갔으며, 2025년에는 전사적 규모로 확산될 전망입니다. 특히 생성형 AI는 가장 빠르게 발전 중입니다. AI 기술은 실용적 에이전트 확산, 검색·증강 기술 진화, 산업별 맞춤형 솔루션, 과도한 AI 현상 해결 등 다양한 분야에 영향을 미치고 있으며, AI의 거버넌스와 규제, AI 주도형 자율주행, 범용 AI 로봇 등도 중요한 이슈입니다. 또한, AI는 누구나 쉽게 쓸 수 있게 발전하며, 대화형 인터페이스는 더욱 직관적이 되고, 고객 경험 분야도 근본적으로 변화하고 있습니다. 앞으로는 기초 모델의 진화와 AI 에이전트의 발전이 두드러지고, 반도체 외 다른 산업에서도 AI 관련 기술이 떠오를 것으로 전망됩니다.


#### 구글 이미지 검색

In [24]:
import pprint
search = GoogleSerperAPIWrapper(type="images")
results = search.results("Lion")
pprint.pp(results)

{'searchParameters': {'q': 'Lion',
                      'gl': 'us',
                      'hl': 'en',
                      'type': 'images',
                      'num': 10,
                      'engine': 'google'},
 'images': [{'title': 'Lion - Wikipedia',
             'imageUrl': 'https://upload.wikimedia.org/wikipedia/commons/a/a6/020_The_lion_king_Snyggve_in_the_Serengeti_National_Park_Photo_by_Giles_Laurent.jpg',
             'imageWidth': 5168,
             'imageHeight': 3448,
             'thumbnailUrl': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR_HGdbfaRDdKJpWDDdD0AkS58dHashCEbqH9yTMz4j7lQIC6iD&s',
             'thumbnailWidth': 275,
             'thumbnailHeight': 183,
             'source': 'Wikipedia, the free encyclopedia',
             'domain': 'en.wikipedia.org',
             'link': 'https://en.wikipedia.org/wiki/Lion',
             'googleUrl': 'https://www.google.com/imgres?imgurl=https%3A%2F%2Fupload.wikimedia.org%2Fwikipedia%2Fcommons%2Fa%2Fa6%2F020

In [25]:
import requests

image_url = results['images'][0]['imageUrl']

res = requests.get(image_url, headers={
    'user-agent': 'Mozilla 5.0'
})
with open('./lion.jpg', 'wb') as f:
    f.write(res.content)

#### 구글 뉴스 검색

In [20]:
search = GoogleSerperAPIWrapper(type="news")
results = search.results("Tesla Inc.")
pprint.pp(results)

{'searchParameters': {'q': 'Tesla Inc.',
                      'gl': 'us',
                      'hl': 'en',
                      'type': 'news',
                      'num': 10,
                      'page': 1,
                      'engine': 'google'},
 'news': [{'title': 'Tesla grants Musk massive pay deal to keep CEO on board '
                    'amid legal battle',
           'link': 'https://www.foxbusiness.com/markets/tesla-grants-musk-massive-pay-deal-keep-ceo-board-amid-legal-battle',
           'section': 'Tesla grants Elon Musk $29 billion in stock options',
           'date': '1 day ago',
           'source': 'Fox Business',
           'position': 1},
          {'title': 'Tesla board awards $30bn of shares to ‘energise and '
                    'focus’ Elon Musk',
           'link': 'https://www.ft.com/content/ae085ae9-3d15-4d68-a28f-39c88e8bf8c8',
           'section': 'Tesla grants Elon Musk $29 billion in stock options',
           'date': '1 day ago',
           'sou

In [26]:
# tbs 파라미터를 활용해 검색 결과의 시간을 제한 가능
# qdr:h (past hour) qdr:d (past day) qdr:w (past week) qdr:m (past month) qdr:y (past year)

# 특정 기간을 설정하는것도 가능
# qdr:h12 (past 12 hours)
# qdr:d3, qdr:w2, qdr:m6, qdr:y2
search = GoogleSerperAPIWrapper(type="news", tbs="qdr:h")
results = search.results("Tesla Inc.")
pprint.pp(results)

{'searchParameters': {'q': 'Tesla Inc.',
                      'gl': 'us',
                      'hl': 'en',
                      'type': 'news',
                      'num': 10,
                      'page': 1,
                      'tbs': 'qdr:h',
                      'engine': 'google'},
 'news': [{'title': 'Tesla: Board announces new pay package for CEO Elon Musk',
           'link': 'https://www.morningstar.com.au/stocks/tesla-board-announces-new-pay-package-ceo-elon-musk',
           'snippet': 'A key provision of the new award is that Musk must '
                      'remain in a senior leadership position at Tesla.',
           'date': '33 minutes ago',
           'source': 'Morningstar',
           'position': 1},
          {'title': 'UK Weighs Tough Choices As Borrowing Rises And Growth '
                    'Slows',
           'link': 'https://finimize.com/content/uk-weighs-tough-choices-as-borrowing-rises-and-growth-slows',
           'snippet': 'British leaders face tax

#### 구글 장소 검색

In [27]:
search = GoogleSerperAPIWrapper(type="places")
results = search.results("Italian restaurants in Upper East Side")
pprint.pp(results)

{'searchParameters': {'q': 'Italian restaurants in Upper East Side',
                      'gl': 'us',
                      'hl': 'en',
                      'type': 'places',
                      'num': 10,
                      'page': 1,
                      'engine': 'google'},
 'places': [{'position': 1,
             'title': "L'Osteria",
             'address': '1219 Lexington Ave, New York, NY 10028',
             'latitude': 40.7771565,
             'longitude': -73.9571345,
             'rating': 4.7,
             'ratingCount': 307,
             'priceLevel': '$30–50',
             'category': 'Italian restaurant',
             'phoneNumber': '(646) 524-6294',
             'website': 'http://www.losterianyc.com/',
             'cid': '13046004590297227899'},
            {'position': 2,
             'title': 'Piccola Cucina Uptown',
             'address': '106 E 60th St, New York, NY 10022',
             'latitude': 40.7632756,
             'longitude': -73.96896149999999,

### 검색 AI Agent 개발

In [28]:
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType

search = GoogleSerperAPIWrapper()

search_tool = Tool(
    name="GoogleSerperAPIWrapper",
    func=search.run,
    description="useful for when you need to ask with search",
)

model = ChatOpenAI(model="gpt-4.1-nano")

agent = initialize_agent(
    tools=[search_tool],
    agent_type=AgentType.OPENAI_FUNCTIONS,
    llm=model,
    verbose=True # Thought process 확인
)

result = agent.invoke({"input": "Make me a detailed report on coldwar. with source link."})
print(result['output'])

C:\Users\USER\AppData\Local\Temp\ipykernel_18728\247644559.py:14: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(




> Entering new AgentExecutor chain...
Thought: I need to gather detailed information about the Cold War, including its causes, major events, key figures, and consequences, along with a credible source link. I will perform a search to find a comprehensive and reliable report or overview.

Action: GoogleSerperAPIWrapper
Action Input: "detailed report on Cold War with source link"
Observation: The purpose of this document collection is to allow students and teachers to develop their own questions and lines of historical enquiry on the Cold War. Cold War Origins "This collection of primary source documents discusses international relations during World War II and the years shortly after. This collection explores the emergence and intensification of the Cold War. It provides sources that help us see how the conflict was framed in its early ... These sources provide details about major Cold War events, highlight individuals who figure prominently in Cold War history, and attest to the inte

In [29]:
print(result['output'])

The Cold War was a period of geopolitical tension between the United States and the Soviet Union that lasted from roughly 1947 to 1991. It was characterized by ideological rivalry, militarization, and political competition without direct large-scale fighting between the superpowers, though numerous proxy wars occurred.

**Causes:**
- Fundamental ideological differences between capitalism (U.S.) and communism (Soviet Union).
- Disagreements over the post-World War II reconstruction and spheres of influence.
- Mutual distrust and security concerns about each other's intentions.
- The spread of nuclear weapons escalating fears of global annihilation.

**Major Events:**
- The Iron Curtain and the division of Europe.
- The Berlin Blockade (1948-1949).
- The Korean War (1950-1953).
- The Cuban Missile Crisis (1962).
- The Vietnam War (1955-1975).
- The Soviet invasion of Afghanistan (1979).
- The fall of the Berlin Wall (1989).

**Key Figures:**
- U.S. Presidents: Harry S. Truman, John F. Ke

### Bing Web Search API (사용 X)

- [Azure](https://azure.microsoft.com/) 회원가입 필요
- 8월 11일 부로 retired 될 예정으로 v7 생성 불가
- grounding with bing search 서비스는 1000 요청당 $35 비용 발생

```txt
BING_SUBSCRIPTION_KEY=search api 리소스 키
BING_SEARCH_URL=https://api.bing.microsoft.com/v7.0/search
```

In [ ]:
from langchain_community.utilities import BingSearchAPIWrapper

search = BingSearchAPIWrapper(k=4)
search_result = search.run("Which BTS members are still serving in the military as of May 2025?")